In [1]:
import pandas as pd
import numpy as np

In [2]:
traffic = pd.read_csv("../dataset/last_data/accident_0_min_before.csv", encoding='cp949')
traffic_5min = pd.read_csv("../dataset/last_data/accident_5_min_before.csv", encoding='cp949')
traffic_10min = pd.read_csv("../dataset/last_data/accident_10_min_before.csv", encoding='cp949')
traffic_15min = pd.read_csv("../dataset/last_data/accident_15_min_before.csv", encoding='cp949')


non_traffic = pd.read_csv("../dataset/last_data/non_accident_0_min_before.csv", encoding='cp949')
non_traffic_5min = pd.read_csv("../dataset/last_data/non_accident_5_min_before.csv", encoding='cp949')
non_traffic_10min = pd.read_csv("../dataset/last_data/non_accident_10_min_before.csv", encoding='cp949')
non_traffic_15min = pd.read_csv("../dataset/last_data/non_accident_15_min_before.csv", encoding='cp949')

In [3]:
# non_traffic에서 랜덤으로 25136개 추출
non_traffic = non_traffic.sample(n=25136, random_state=42)
non_traffic_5min = non_traffic_5min.sample(n=25136, random_state=42)
non_traffic_10min = non_traffic_10min.sample(n=25136, random_state=42)
non_traffic_15min = non_traffic_15min.sample(n=25136, random_state=42)

In [4]:
static_traffic = traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]
static_non_traffic = non_traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]

In [5]:
drop_traffic_data = traffic.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera' ,'label'], axis=1)
drop_non_traffic_data = non_traffic.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera', 'label'], axis=1)

drop_traffic_data_5 = traffic_5min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)
drop_non_traffic_data_5 = non_traffic_5min.drop([ 'datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)

drop_traffic_data_10 = traffic_10min.drop(['datetime', 'link', 'lanes','speed_limit','length' ,'bump', 'camera','label'], axis=1)
drop_non_traffic_data_10 = non_traffic_10min.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

drop_traffic_data_15 = traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)
drop_non_traffic_data_15 = non_traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

traffic_label = traffic['label']
non_traffic_label = non_traffic['label']

In [6]:
dynamic_traffic_5 = drop_traffic_data_5.values
dynamic_traffic_10 = drop_traffic_data_10.values
dynamic_traffic_15 = drop_traffic_data_15.values

dynamic_non_traffic_5 = drop_non_traffic_data_5.values
dynamic_non_traffic_10 = drop_non_traffic_data_10.values
dynamic_non_traffic_15 = drop_non_traffic_data_15.values

traffic_label = traffic_label.values
non_traffic_label = non_traffic_label.values

static_traffic = static_traffic.values
static_non_traffic = static_non_traffic.values

In [7]:
X, Y = [], []
cnt = 0
for accident in [dynamic_traffic_15, dynamic_traffic_10, dynamic_traffic_5]:
    X.append([])
    Y.append([])
    for info in range(len(dynamic_traffic_15)):
        X[cnt].append(accident[info])
        Y[cnt].append(traffic_label[info])
    cnt += 1

cnt = 0
for non_accident in [dynamic_non_traffic_15, dynamic_non_traffic_10, dynamic_non_traffic_5]:
    for info in range(len(dynamic_non_traffic_15)):
        X[cnt].append(non_accident[info])
        Y[cnt].append(non_traffic_label[info])
    X[cnt] = np.array(X[cnt])
    Y[cnt] = np.array(Y[cnt])
    cnt += 1

In [8]:
from sklearn.model_selection import train_test_split

def split_data(data, label):

    X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=40, shuffle=True)
    X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=40, shuffle=True)

    return X_train, X_test, X_val, Y_train, Y_test, Y_val

/opt/anaconda3/envs/myenv2/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
X_train = []
X_test = []
X_val = []
Y_train = []
Y_test = []
Y_val = []

for sp in range(3):
    data = split_data(X[sp], Y[sp])
    X_train.append(data[0])
    X_test.append(data[1])
    X_val.append(data[2])
    Y_train.append(data[3])
    Y_test.append(data[4])
    Y_val.append(data[5])

### i == 0 15min before
### i == 1 10min before
### i == 2 5min before

In [10]:
for i in range(3):
    X_train[i] = X_train[i].reshape(X_train[i].shape[0], 1, X_train[i].shape[1])
    X_test[i] = X_test[i].reshape(X_test[i].shape[0], 1, X_test[i].shape[1])
    X_val[i] = X_val[i].reshape(X_val[i].shape[0], 1, X_val[i].shape[1])

### Static Feature Split

In [11]:
sX, sY = [], []

for info in range(len(static_traffic)):
    sX.append(static_traffic[info])
    # sY.append(1) == traffic accident occured
    sY.append(1)

for info in range(len(static_non_traffic)):
    sX.append(static_non_traffic[info])
    # sY.append(0) == traffic accident not occured
    sY.append(0)

sX = np.array(sX)
sY = np.array(sY)

sX_train, sX_test, sY_train, sY_test = train_test_split(sX, sY, test_size=0.2, random_state=40, shuffle=True)
sX_val, sX_test, sY_val, sY_test = train_test_split(sX_test, sY_test, test_size=0.5, random_state=40, shuffle=True)

In [12]:
Y_train[0] = Y_train[0].astype(float).reshape((-1,1))
Y_val[0] = Y_val[0].astype(float).reshape((-1,1))
Y_test[0] = Y_test[0].astype(float).reshape((-1,1))

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, concatenate, Flatten, Reshape
from tensorflow.keras.models import Model

In [49]:
def LS_DS(ls, ls_ds, ds):
    input_5 = Input(shape=(1, 11))
    input_10 = Input(shape=(1, 11))
    input_15 = Input(shape=(1, 11))

    input_st = Input(shape=(5,))

    lstm_1 = LSTM(ls, return_sequences=False)(input_5)
    dense_1 = Dense(ls_ds, activation='relu')(lstm_1)

    lstm_2 = LSTM(ls, return_sequences=False)(input_10)
    dense_2 = Dense(ls_ds, activation='relu')(lstm_2)

    lstm_3 = LSTM(ls, return_sequences=False)(input_15)
    dense_3 = Dense(ls_ds, activation='relu')(lstm_3)

    con_1 = concatenate([dense_1, dense_2, dense_3])

    lstm_reshape = Reshape((-1, 30))(con_1)
    st_reshape = Reshape((-1, sX.shape[1]))(input_st)
    combine = concatenate([lstm_reshape, st_reshape])


    dense_4 = Dense(ds, activation='relu')(combine)
    dense_5 = Dense(ds, activation='relu')(dense_4)
    out = Dense(1, activation='sigmoid')(dense_5)

    m = Model(inputs=[input_15, input_10, input_5, input_st], outputs=out)
    m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.BinaryAccuracy(threshold=0.5)])
    
    return m

In [50]:
m = LS_DS(32, 10, 50)

hist = m.fit([X_train[0], X_train[1], X_train[2], sX_train], Y_train[0], epochs=30, batch_size=32, validation_data=([X_val[0], X_val[1], X_val[2], sX_val], Y_val[0]), verbose=1)

loss, acc, auc, precision, recall, binary_accuracy = m.evaluate([X_test[0], X_test[1], X_test[2], sX_test], Y_test[0], verbose=0)
print("m parameter", m.count_params())
print('loss : ', loss)
print('acc : ', acc)
print('auc : ', auc)
print('precision : ', precision)
print('recall : ', recall)
print('binary_accuracy : ', binary_accuracy)
#print f1-score with calculate recall precision

f1_score = (2 * precision * recall) / (precision + recall)
print('f1_score : ', f1_score)

Epoch 1/30
1257/1257 [==============================] - 5s 2ms/step - loss: 0.5667 - accuracy: 0.6904 - auc: 0.7689 - precision_13: 0.6970 - recall_13: 0.6752 - binary_accuracy: 0.6904 - val_loss: 0.5424 - val_accuracy: 0.7145 - val_auc: 0.7949 - val_precision_13: 0.7658 - val_recall_13: 0.6209 - val_binary_accuracy: 0.7145
Epoch 2/30
1257/1257 [==============================] - 2s 1ms/step - loss: 0.5399 - accuracy: 0.7143 - auc: 0.7959 - precision_13: 0.7400 - recall_13: 0.6619 - binary_accuracy: 0.7143 - val_loss: 0.5399 - val_accuracy: 0.7223 - val_auc: 0.7993 - val_precision_13: 0.7809 - val_recall_13: 0.6205 - val_binary_accuracy: 0.7223
Epoch 3/30
1257/1257 [==============================] - 2s 1ms/step - loss: 0.5352 - accuracy: 0.7202 - auc: 0.8006 - precision_13: 0.7445 - recall_13: 0.6718 - binary_accuracy: 0.7202 - val_loss: 0.5366 - val_accuracy: 0.7235 - val_auc: 0.8014 - val_precision_13: 0.7202 - val_recall_13: 0.7339 - val_binary_accuracy: 0.7235
Epoch 4/30
1257/1257 [